The original data is available at:   
- /glade/p/cesm/cseg//inputdata/lnd/clm2/surfdata/surfdata_0.9x1.25_simyr1850_c110921.nc    

The processed data is available at:    
- /glade/scratch/zhonghua/urban_params/urban_surface.nc

Please check `0_CESM1_surface_data_EDA.ipynb` for more information

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
p = "/glade/p/cesm/cseg//inputdata/lnd/clm2/surfdata/surfdata_0.9x1.25_simyr1850_c110921.nc" # CESM 1
save_path = "/glade/scratch/zhonghua/urban_params/urban_surface.nc" 

var_ls = [
    "CANYON_HWR","EM_IMPROAD","EM_PERROAD","EM_ROOF","EM_WALL",
    "HT_ROOF","THICK_ROOF","THICK_WALL","T_BUILDING_MAX","T_BUILDING_MIN",
    "WTLUNIT_ROOF","WTROAD_PERV","ALB_IMPROAD","ALB_PERROAD","ALB_ROOF","ALB_WALL",
    "TK_ROOF","TK_WALL","TK_IMPROAD","CV_ROOF","CV_WALL","CV_IMPROAD",
    "NLEV_IMPROAD","PCT_URBAN"
]

ds = xr.open_dataset(p)[var_ls]

d = {} # d[('lsmlat','lsmlon')] = ['CANYON_HWR','EM_IMPROAD',...]
for var in ds:
    if ds[var].dims in d:
        d[ds[var].dims].append(var)
    else:
        d[ds[var].dims] = [var]
        
for k in d:
    print(k,":",len(d[k]))

('lsmlat', 'lsmlon') : 14
('numsolar', 'numrad', 'lsmlat', 'lsmlon') : 4
('nlevurb', 'lsmlat', 'lsmlon') : 6


In [3]:
# 2D
ds_2d = ds[d[('lsmlat', 'lsmlon')]]
# 4D
ds_4d = ds[d[('numsolar', 'numrad', 'lsmlat', 'lsmlon')]].sel(numsolar=0,numrad=0)
# 3D
ds_3d_a = ds[["TK_ROOF","TK_WALL","CV_ROOF","CV_WALL"]].sel(nlevurb=0)
ds_3d_b_0 = ds[["TK_IMPROAD","CV_IMPROAD"]].sel(nlevurb=0)\
.rename({"TK_IMPROAD":"TK_IMPROAD_0","CV_IMPROAD":"CV_IMPROAD_0"})
ds_3d_b_1 = ds[["TK_IMPROAD","CV_IMPROAD"]].sel(nlevurb=1)\
.rename({"TK_IMPROAD":"TK_IMPROAD_1","CV_IMPROAD":"CV_IMPROAD_1"})

In [4]:
ds_final = xr.merge([ds_2d,ds_4d,ds_3d_a,ds_3d_b_0,ds_3d_b_1])
ds_final.to_netcdf(save_path)

In [5]:
ds_l = xr.open_dataset(save_path)
ds_l

<xarray.Dataset>
Dimensions:         (lsmlat: 192, lsmlon: 288)
Dimensions without coordinates: lsmlat, lsmlon
Data variables: (12/26)
    CANYON_HWR      (lsmlat, lsmlon) float64 ...
    EM_IMPROAD      (lsmlat, lsmlon) float64 ...
    EM_PERROAD      (lsmlat, lsmlon) float64 ...
    EM_ROOF         (lsmlat, lsmlon) float64 ...
    EM_WALL         (lsmlat, lsmlon) float64 ...
    HT_ROOF         (lsmlat, lsmlon) float64 ...
    ...              ...
    CV_ROOF         (lsmlat, lsmlon) float64 ...
    CV_WALL         (lsmlat, lsmlon) float64 ...
    TK_IMPROAD_0    (lsmlat, lsmlon) float64 ...
    CV_IMPROAD_0    (lsmlat, lsmlon) float64 ...
    TK_IMPROAD_1    (lsmlat, lsmlon) float64 ...
    CV_IMPROAD_1    (lsmlat, lsmlon) float64 ...
Attributes: (12/24)
    Conventions:                        NCAR-CSM
    History_Log:                        created on: 10-06-09 12:38:50
    Logname:                            erik
    Host:                               be1005en.ucar.edu
    Source:                             Community Land Model: CLM3
    Version:                            $HeadURL: https://svn-ccsm-models.cgd...
    ...                                 ...
    Fmax_raw_data_file_name:            mksrf_fmax.070406.nc
    Urban_raw_data_file_name:           mksrf_urban_3den_0.5x0.5_simyr2000.c0...
    Irrig_raw_data_file_name:           
    Lai_raw_data_file_name:             mksrf_lai_global_c090506.nc
    history:                            Wed Sep 21 17:01:03 2011: ncks -A -v ...
    NCO:                                20110921